<a href="https://colab.research.google.com/github/S-Anupama/Protos_chatbot/blob/main/protos_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rasa

In [2]:
!pip install nest-asyncio

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
!python --version

Python 3.10.12


In [5]:
!rasa init --no-prompt

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
(0lqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqk(B
(0x(B Rasa Open Source reports anonymous usage telemetry to help improve the product (0x(B
(0x(B for all its users.                                                             (0x(B
(0x(B                                                                                (0x(B
(0x(B If you'd like to opt-out, you can use `ra

In [6]:
!pip install transformers
!pip install torch

In [7]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="cardiffnlp/twitter-roberta-base-sentiment-latest")
#example usage
text = "I love this!"
result = sentiment_analysis(text)[0]

label = result['label']

if label=='LABEL_0':
  label='NEGATIVE'
elif label=='LABEL_1':
  label='NEUTRAL'
elif label=='LABEL_2':
  label='POSITIVE'

  print(f"Label: {label}")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
import os

# Create directories for Rasa project
os.makedirs("chatbot_project", exist_ok=True)
os.chdir("chatbot_project")

# Create necessary subdirectories
os.makedirs("data", exist_ok=True)
os.makedirs("actions", exist_ok=True)


In [10]:
#create domain.yml

with open("domain.yml", "w") as f:
    f.write("""
    version: "3.1"
    name: "Protos"
    intents:
      - greet
      - sentiment_check
      - goodbye
    responses:
      utter_greet:
        - text: "Hello! How can I assist you today?"
      utter_goodbye:
        - text: "Goodbye! Have a great day!"
      utter_positive:
        - text: "It seems like you're feeling positive!"
      utter_neutral:
        - text: "It seems like you're feeling neutral."
      utter_negative:
        - text: "It seems like you're feeling negative."
    actions:
      - action_analyze_sentiment
    """)


In [11]:
# creating stories.yml

with open("data/stories.yml", "w") as f:
    f.write("""
    version: "3.1"
    stories:
    - story: greet and check sentiment
      steps:
      - intent: greet
      - action: utter_greet
      - intent: sentiment_check
      - action: action_analyze_sentiment
      - action: utter_positive
    - story: farewell
      steps:
      - intent: goodbye
      - action: utter_goodbye
    """)


In [12]:
# Create the config.yml file for Rasa
with open("config.yml", "w") as f:
    f.write("""
version: "3.1"

language: en

pipeline:
  - name: "WhitespaceTokenizer"
  - name: "RegexFeaturizer"
  - name: "LexicalSyntacticFeaturizer"
  - name: "CountVectorsFeaturizer"
  - name: "DIETClassifier"
    epochs: 100
    constrain_similarities: true
  - name: "ResponseSelector"
    epochs: 100

policies:
  - name: "MemoizationPolicy"
  - name: "RulePolicy"
  - name: "TEDPolicy"
    max_history: 5
    epochs: 100


""")




In [13]:
#action.py

with open("actions/actions.py", "w") as f:
  f.write("""

from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
from transformers import pipeline

class ActionAnalyzeSentiment(Action):

  def name(self) -> str:
    return "action_analyze_sentiment"

  def run(self, dispatcher: CollectingDispatcher,
          tracker:Tracker,
          domain: dict) -> list:

   print("Hey there! It's Protos, your personal sentiment anaylsis bot")

#get user meassage
    message = tracker.latest_message['text']

    print(f"You: {message}")

    sentiment_analysis = pipeline("sentiment-analysis",model = "cardiffnlp/twitter-roberta-base-sentiment-latest")
    result = sentiment_analysis(message)[0]

    label = result['label']
    if label == 'LABEL_0':
      label = 'NEGATIVE'
    elif label == 'LABEL_1':
      label = 'NEUTRAL'
    elif label == 'LABEL_2':
      label = 'POSITIVE'

    if label == "POSITIVE":
      response = "It seems like you're feeling positive."
    elif label == "NEUTRAL":
      response = "It seems like you're feeling neutral."
    elif label == "NEGATIVE":
      response = "It seems like you're feeling negative."

      print(f"Protos: {response}")

    dispatcher.utter_message(text=response)

    return[]

  """)






In [14]:
!rasa train

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/validation.py:134: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3121: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as spe

In [16]:
!rasa run actions

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/validation.py:134: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3121: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as spe

In [17]:
!rasa shell

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/validation.py:134: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3121: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as spe

In [19]:
#requirements.txt
# Create the chatbot_project directory first if it doesn't exist
import os

os.makedirs("chatbot_project", exist_ok=True)

with open("chatbot_project/requirements.txt", "w") as f:
    f.write("""
torch
transformers
rasa
rasa-sdk
nest-asyncio
""")

In [23]:
#procfile
with open("chatbot_project/Procfile", "w") as f:
    f.write("web: rasa run --enable-api --cors \"*\" --port $PORT\nworker: rasa run actions")


 #runtime.txt
with open("chatbot_project/runtime.txt", "w") as f:
    f.write("python-3.10")


In [24]:
import shutil

shutil.make_archive("chatbot_project", 'zip', "chatbot_project")


'/content/chatbot_project/chatbot_project/chatbot_project.zip'

In [26]:
from google.colab import files

files.download("chatbot_project.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>